In [34]:
import pandas as pd
from scipy.io import arff
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import confusion_matrix
import seaborn as sn
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from pyclustering.cluster.xmeans import xmeans, splitting_type
from imblearn.over_sampling import SMOTE
from implicitModel import Implicit_ME


In [35]:
def DecisionTree(X_train,y_train,X_test,y_test):
    clf = DecisionTreeClassifier(criterion = "entropy", splitter = "random").fit(X_train,y_train)
    y_pred_test = clf.predict(X_test)
    
    #matrices
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_test)
    auc = metrics.roc_auc_score(y_test,  y_pred_test)
    result = {
      "Accuracy": metrics.accuracy_score(y_test, y_pred_test),
      "F1_score": f1_score(y_test, y_pred_test, average='macro'),
      "AUC_score": auc,
      "Prediction": y_pred_test
      
    }
    return result

In [36]:
def SVM(X_train,y_train,X_test,y_test):
    clf = svm.SVC().fit(X_train,y_train)
    y_pred_test = clf.predict(X_test)
    
    #matrices
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_test)
    auc = metrics.roc_auc_score(y_test,  y_pred_test)
    result = {
      "Accuracy": metrics.accuracy_score(y_test, y_pred_test),
      "F1_score": f1_score(y_test, y_pred_test, average='macro'),
      "AUC_score": auc,
      "Prediction": y_pred_test
      
    }
    return result

In [37]:
def Logistic(X_train,y_train,X_test,y_test):
    clf = LogisticRegression(max_iter=100,penalty = 'none').fit(X_train,y_train)
    y_pred_test = clf.predict(X_test)
    
    #matrices
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_test)
    auc = metrics.roc_auc_score(y_test,  y_pred_test)
    result = {
      "Accuracy": metrics.accuracy_score(y_test, y_pred_test),
      "F1_score": f1_score(y_test, y_pred_test, average='macro'),
      "AUC_score": auc,
      "Prediction": y_pred_test
      
    }
    return result

In [38]:
def gate_ntw(X_train,y_train,X_test,y_test):
    gateDT = DecisionTreeClassifier(criterion = "entropy", splitter = "best").fit(X_train,y_train)
    gateSVM = svm.SVC(probability=True).fit(X_train,y_train)
    gateLOG = LogisticRegression(max_iter=10000).fit(X_train,y_train)
    voting_clf = VotingClassifier(
    estimators=[('DecisionTree',gateDT), ('SVM',gateSVM),('Logistic',gateLOG)],voting='soft')
    voting_clf.fit(X_train, y_train)
    final_predictions = voting_clf.predict(X_test)
    print("Accuracy of Mixture of Experts:",metrics.accuracy_score(y_test, final_predictions))
    print("F1 score", f1_score(y_test,final_predictions, average='macro'))
    print("AUC score",  metrics.roc_auc_score(y_test,  final_predictions))
    

In [39]:
data = arff.loadarff('../dataSet/JIRA/activemq-5.0.0.arff')
df = pd.DataFrame(data[0])
X= df.iloc[ : , :-1].values
y= df['RealBugCount'].apply(lambda x : 1 if(x > 0) else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [40]:
experts = []
experts.append(DecisionTree)
experts.append(SVM)
experts.append(Logistic)
Implicit_ME(X_train,y_train,X_test,y_test,10,experts,gate_ntw)

C:\Users\AdityaShankar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\AdityaShankar\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

Accuracy of Mixture of Experts: 0.8938992042440318
F1 score 0.7694189602446483
AUC score 0.742845394736842
